In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, when
import pyspark.sql.types as st

import csv
from bisect import bisect, bisect_left, bisect_right

import datetime
# spark = SparkSession(sc)
# sqlContext = SQLContext(sc)

In [2]:
geo_file =  'nyc_cscl.csv'

In [3]:
sc = SparkContext()
sqlContext = SQLContext(sc)
directory = 'hdfs:///data/share/bdm/nyc_parking_violations/'

#rdd2 = sc.textFile(geo_file).mapPartitionsWithIndex(parseCL)

In [4]:
def parseCL(idx,part):
    if idx == 0:
        next(part)
    for p in csv.reader(part):
        
        #Left Houses
        if '-' in p[2] and '-' in p[3]:#need and statement otherwise may run into case with nulls
            V1_LL, V2_LL = p[2].strip().split('-')
            LL_HN = V1_LL#(V1_LL, V2_LL)
            
            V1_LH, V2_LH = p[3].strip().split('-')
            LH_HN = V1_LH#(V1_LH, V2_LH)
            
        elif len(p[2])==0  and len(p[3])==0:
            LL_HN, LH_HN = '0', '0'
            
        else:
            LL_HN, LH_HN = p[2].split('-')[0],  p[3].split('-')[0]
            
        #Right Houses
        if '-' in p[4] and '-' in p[5]:
            V1_RL, V2_RL = p[4].strip().split('-')
            RL_HN = V1_RL#(V1_RL, V2_RL)
            
            V1_RH, V2_RH = p[5].strip().split('-')
            RH_HN = V1_RH#(V1_RH, V2_RH)
            
        elif len(p[4])==0 and len(p[5])==0:
            RL_HN, RH_HN = '0', '0'
            
        else:
            RL_HN, RH_HN = p[4].split('-')[0], p[5].split('-')[0]
        
        #ID
        if len(p[0])==0:
            continue
        else:
            ID = p[0].strip()
        
        #Full_Street and Street Label:
        if len(p[28])==0 and len(p[10])==0:
            continue
        else:
            full_st = p[28].lower().strip()
            st_label = p[10].lower().strip()
        
        #borocode
        if p[13].strip() not in ['1','2','3','4','5']:
            continue
        borocode = p[13]
        
        #yield
        row = (ID, full_st, st_label, borocode, 
              LL_HN, LH_HN, RL_HN, RH_HN )
        
        yield row

In [5]:
labels=('ID','Full Street','st label','borocode','LL_HN','LH_HN','RL_HN','RH_HN')

centerLine = sc.textFile(geo_file).mapPartitionsWithIndex(parseCL,labels)
print(centerLine.count())

CL_frame = sqlContext.createDataFrame(centerLine,labels)
centerLine.take(5)
CL_frame.show(15)

547313
+------+--------------------+--------------------+--------+-----+-----+-----+-----+
|    ID|         Full Street|            st label|borocode|LL_HN|LH_HN|RL_HN|RH_HN|
+------+--------------------+--------------------+--------+-----+-----+-----+-----+
|164809|mitsubishi wild w...|mitsubishi wild w...|       2|    0|    0|    0|    0|
|  6110|              28 ave|               28 av|       4|  215|  215|  215|  215|
|145494|     schermerhorn st|     schermerhorn st|       3|  317|  399|  316|  360|
| 61140|       arlington ave|        arlington av|       2| 5631| 5699| 5602| 5698|
| 12438|         queens blvd|         queens blvd|       4|  120|  120|    0|    0|
| 15432|       arthur kil rd|      arthur kill rd|       5|  555|  555|  528|  554|
| 52620|           e  181 st|            e 181 st|       2|  731|  759|  730|  758|
| 83272|               58 ln|               58 la|       4|   51|   51|   51|   51|
| 26405|          forest ave|           forest av|       4|   68|   6

In [8]:
def parseCSV(idx,part):  
    county = {
        'BROOKLYN':('BK','K','KING','KINGS'),
        'MANHATTAN':('MAN','MH','MN','NEWY','NEW Y','NY'),
        'BRONX':('BRONX','BX'), #MATCH ON SUBSTRING
        'QUEENS': ('Q','QN','QNS','QU','QUEEN'),
        'STATEN ISLAND': ('R','RICHMOND')
    }

    counties ={}
    boros = list(county.keys())
    boros = [boro.lower() for boro in county.keys()]

    county_tups = list(county.values())
    for code in county_tups:
        for symb in code:
            counties[symb.lower()]=str(county_tups.index(code)+1)
        
    years = ['2015','2016','2017','2018','2019']
    
    if idx == 0:
        next(part)
    for p in csv.reader(part):
        if p[23].isalpha() and p[24]=='' and p[21]=='' and p[23]=='':
            continue
    
        if '-' in p[23]:
            HN = p[23].split('-')[0].lower().strip()
            HNC = p[23].split('-')[1].lower().strip()
        
        elif '' == p[23]:
            continue
        else:
            HN = p[23].lower().strip()
            HNC = ""
        SN = p[24].lower().strip()
        
        cn = p[21].lower().strip()
        if cn not in list(counties.keys()):
            next(part)
        try:    
            CN = counties[cn]
        except:
            continue
        
        year = p[4][-4:].lower().strip()
        
        #yield result
        row = (HN,HNC,SN,CN,year)
        
        if year in years:
            yield row

In [9]:
rows = sc.textFile('Parking_Violations_Issued_201[5-9]_simplified.csv').mapPartitionsWithIndex(parseCSV)
print(rows.count())


tkts_Frame = sqlContext.createDataFrame(rows,('House Number','HN Compound','Street Name', 'County','Year'))
tkts_Frame.show(40)


17244
+------------+-----------+-----------------+------+----+
|House Number|HN Compound|      Street Name|County|Year|
+------------+-----------+-----------------+------+----+
|         325|           |       e 112th st|     2|2018|
|         410|           |       e 100th st|     2|2018|
|         283|           |       4th avenue|     1|2018|
|        2790|           |          8th ave|     2|2018|
|        2790|           |          8th ave|     2|2018|
|          20|           |          pine st|     2|2018|
|           5|           |         st marks|     2|2018|
|         150|           |          w 21 st|     2|2018|
|          54|           |          w 47 st|     2|2018|
|          54|           |          w 47 st|     2|2018|
|          50|           |          w 47 st|     2|2018|
|          50|           |          w 47 st|     2|2018|
|          14|           |          w 47 st|     2|2018|
|         490|           |   longfellow ave|     3|2018|
|          12|         35

In [10]:
tkts_Frame = tkts_Frame.withColumn('House Number',tkts_Frame['House Number'].cast('int'))
tkts_Frame = tkts_Frame.withColumn('HN Compound',tkts_Frame['HN Compound'].cast('int'))
tkts_Frame.show(25)
tkts_Frame.count()

+------------+-----------+--------------+------+----+
|House Number|HN Compound|   Street Name|County|Year|
+------------+-----------+--------------+------+----+
|         325|       null|    e 112th st|     2|2018|
|         410|       null|    e 100th st|     2|2018|
|         283|       null|    4th avenue|     1|2018|
|        2790|       null|       8th ave|     2|2018|
|        2790|       null|       8th ave|     2|2018|
|          20|       null|       pine st|     2|2018|
|           5|       null|      st marks|     2|2018|
|         150|       null|       w 21 st|     2|2018|
|          54|       null|       w 47 st|     2|2018|
|          54|       null|       w 47 st|     2|2018|
|          50|       null|       w 47 st|     2|2018|
|          50|       null|       w 47 st|     2|2018|
|          14|       null|       w 47 st|     2|2018|
|         490|       null|longfellow ave|     3|2018|
|          12|         35|        35 ave|     4|2018|
|        2920|       null|  

17244

In [11]:
tkts_HN_even = tkts_Frame.filter(tkts_Frame['House Number']%2==0)

tkts_HN_odd = tkts_Frame.filter(tkts_Frame['House Number']%2==1)

In [12]:
tkts_HN_even.show(15)
tkts_HN_odd.show(15)

+------------+-----------+--------------+------+----+
|House Number|HN Compound|   Street Name|County|Year|
+------------+-----------+--------------+------+----+
|         410|       null|    e 100th st|     2|2018|
|        2790|       null|       8th ave|     2|2018|
|        2790|       null|       8th ave|     2|2018|
|          20|       null|       pine st|     2|2018|
|         150|       null|       w 21 st|     2|2018|
|          54|       null|       w 47 st|     2|2018|
|          54|       null|       w 47 st|     2|2018|
|          50|       null|       w 47 st|     2|2018|
|          50|       null|       w 47 st|     2|2018|
|          14|       null|       w 47 st|     2|2018|
|         490|       null|longfellow ave|     3|2018|
|          12|         35|        35 ave|     4|2018|
|        2920|       null|   w 21 street|     1|2018|
|        2920|       null|   w 21 street|     1|2018|
|        2790|       null|       8th ave|     2|2018|
+------------+-----------+--

In [13]:
#####EVEN Conditions ########
cond1 = CL_frame['Full Street'] == tkts_HN_even['Street Name']
cond2 = CL_frame['st label'] == tkts_HN_even['Street Name']
cond3 = CL_frame['borocode'] == tkts_HN_even['County']
cond4 = CL_frame['RH_HN'] >= tkts_HN_even['House Number']
cond5 = CL_frame['RL_HN'] <= tkts_HN_even['House Number']

#####ODD Conditions ########
cond6 = CL_frame['Full Street'] == tkts_HN_odd['Street Name']
cond7 = CL_frame['st label'] == tkts_HN_odd['Street Name']
cond8 = CL_frame['borocode'] == tkts_HN_odd['County']
cond9 = CL_frame['LH_HN'] >= tkts_HN_odd['House Number']
cond10 = CL_frame['LL_HN'] <= tkts_HN_odd['House Number']

In [14]:
df_match_3145 = CL_frame.join(tkts_HN_even,cond3 & cond1 & cond4 & cond5 )
df_match_3145.show(15)
a = df_match_3145.count()
a

+-----+------------+------------+--------+-----+-----+-----+-----+------------+-----------+------------+------+----+
|   ID| Full Street|    st label|borocode|LL_HN|LH_HN|RL_HN|RH_HN|House Number|HN Compound| Street Name|County|Year|
+-----+------------+------------+--------+-----+-----+-----+-----+------------+-----------+------------+------+----+
|37719|  3rd avenue|        3 av|       3|    0|    0| 4500| 4598|        4574|       null|  3rd avenue|     3|2018|
|23058|ditmars blvd|ditmars blvd|       4|   36|   36|   36|   36|          36|         19|ditmars blvd|     4|2015|
|10092|ditmars blvd|ditmars blvd|       4|    0|    0|   94|   94|          94|          0|ditmars blvd|     4|2015|
|23052|ditmars blvd|ditmars blvd|       4|   28|   28|   28|   28|          28|         18|ditmars blvd|     4|2015|
|23060|ditmars blvd|ditmars blvd|       4|   38|   39|   38|   39|          38|          5|ditmars blvd|     4|2017|
|12761|ditmars blvd|ditmars blvd|       4|  103|  104|  102|  10

2578

In [19]:
df_match_3245 = CL_frame.join(tkts_HN_even,cond3 & cond2 & cond4 & cond5 )
df_match_3245.show(15)
b = df_match_3245.count()
b

+-----+--------------+--------+--------+-----+-----+-----+-----+------------+-----------+-----------+------+----+
|   ID|   Full Street|st label|borocode|LL_HN|LH_HN|RL_HN|RH_HN|House Number|HN Compound|Street Name|County|Year|
+-----+--------------+--------+--------+-----+-----+-----+-----+------------+-----------+-----------+------+----+
|72726|      e  59 st| e 59 st|       1|  325|  399|  326|  398|         364|       null|    e 59 st|     1|2018|
|72726|         59 st| e 59 st|       1|  325|  399|  326|  398|         364|       null|    e 59 st|     1|2018|
|72726|       east 59| e 59 st|       1|  325|  399|  326|  398|         364|       null|    e 59 st|     1|2018|
|72726|     east 59th| e 59 st|       1|  325|  399|  326|  398|         364|       null|    e 59 st|     1|2018|
|72726|    e 59 th st| e 59 st|       1|  325|  399|  326|  398|         364|       null|    e 59 st|     1|2018|
|72726|     e 59th st| e 59 st|       1|  325|  399|  326|  398|         364|       null

3591

In [20]:
df_match_869_10 = CL_frame.join(tkts_HN_odd,cond8 & cond6 & cond9 & cond10 )
df_match_869_10.show(15)
c = df_match_869_10.count()
c

+------+--------------+--------------+--------+-----+-----+-----+-----+------------+-----------+--------------+------+----+
|    ID|   Full Street|      st label|borocode|LL_HN|LH_HN|RL_HN|RH_HN|House Number|HN Compound|   Street Name|County|Year|
+------+--------------+--------------+--------+-----+-----+-----+-----+------------+-----------+--------------+------+----+
| 23057|  ditmars blvd|  ditmars blvd|       4|   35|   35|   35|   35|          35|         19|  ditmars blvd|     4|2018|
| 23057|  ditmars blvd|  ditmars blvd|       4|   35|   35|   35|   35|          35|          3|  ditmars blvd|     4|2018|
| 23054|  ditmars blvd|  ditmars blvd|       4|   31|   31|   31|   31|          31|         12|  ditmars blvd|     4|2018|
| 23053|  ditmars blvd|  ditmars blvd|       4|   29|   30|   29|   30|          29|         24|  ditmars blvd|     4|2016|
| 23055|  ditmars blvd|  ditmars blvd|       4|   31|   32|   31|   32|          31|         12|  ditmars blvd|     4|2018|
| 20466|

2790

In [21]:
df_match_879_10 = CL_frame.join(tkts_HN_odd,cond8 & cond7 & cond9 & cond10 )
df_match_879_10.show(15)
d = df_match_879_10.count()
d

+-----+-----------------+------------+--------+-----+-----+-----+-----+------------+-----------+------------+------+----+
|   ID|      Full Street|    st label|borocode|LL_HN|LH_HN|RL_HN|RH_HN|House Number|HN Compound| Street Name|County|Year|
+-----+-----------------+------------+--------+-----+-----+-----+-----+------------+-----------+------------+------+----+
|23057|     ditmars blvd|ditmars blvd|       4|   35|   35|   35|   35|          35|         19|ditmars blvd|     4|2018|
|23057|     ditmars blvd|ditmars blvd|       4|   35|   35|   35|   35|          35|          3|ditmars blvd|     4|2018|
|23054|     ditmars blvd|ditmars blvd|       4|   31|   31|   31|   31|          31|         12|ditmars blvd|     4|2018|
|23053|     ditmars blvd|ditmars blvd|       4|   29|   30|   29|   30|          29|         24|ditmars blvd|     4|2016|
|23055|     ditmars blvd|ditmars blvd|       4|   31|   32|   31|   32|          31|         12|ditmars blvd|     4|2018|
|23057|        ditmars b

4103

# ~55million total, 35M matched

In [22]:
total = a + b + c + d
total


13062

In [23]:
grouped_even = df_match_3245.union(df_match_3145)
grouped_odd = df_match_869_10.union(df_match_879_10)

grouped = grouped_odd.union(grouped_even)

In [24]:
grouped.show(50)

+------+--------------+--------------+--------+-----+-----+-----+-----+------------+-----------+--------------+------+----+
|    ID|   Full Street|      st label|borocode|LL_HN|LH_HN|RL_HN|RH_HN|House Number|HN Compound|   Street Name|County|Year|
+------+--------------+--------------+--------+-----+-----+-----+-----+------------+-----------+--------------+------+----+
| 23057|  ditmars blvd|  ditmars blvd|       4|   35|   35|   35|   35|          35|         19|  ditmars blvd|     4|2018|
| 23057|  ditmars blvd|  ditmars blvd|       4|   35|   35|   35|   35|          35|          3|  ditmars blvd|     4|2018|
| 23054|  ditmars blvd|  ditmars blvd|       4|   31|   31|   31|   31|          31|         12|  ditmars blvd|     4|2018|
| 23053|  ditmars blvd|  ditmars blvd|       4|   29|   30|   29|   30|          29|         24|  ditmars blvd|     4|2016|
| 23055|  ditmars blvd|  ditmars blvd|       4|   31|   32|   31|   32|          31|         12|  ditmars blvd|     4|2018|
| 20466|

In [25]:
grouped.count()

13062

In [163]:
group = grouped.groupby(['ID',"Year"]).count()
group.show()
group.count()

+------+----+-----+
|    ID|Year|count|
+------+----+-----+
|  6365|2018|    2|
| 30485|2018|    3|
|  9328|2015|   14|
| 33645|2018|    4|
| 28412|2018|    4|
| 63150|2018|    1|
| 22631|2016|    2|
|183561|2016|    1|
| 33777|2017|    1|
| 16522|2018|    1|
| 31216|2018|    2|
| 23145|2017|    2|
|  5115|2016|   12|
|101512|2018|    1|
|  6493|2018|    1|
|102832|2016|    2|
| 93274|2018|    1|
| 49359|2015|    1|
| 80909|2016|    1|
|122144|2015|    4|
+------+----+-----+
only showing top 20 rows



3678

# Summary Stats

In [162]:
uniq_id = group.count()
uniq_id

3678

In [169]:
counted = group.groupby(['ID']).pivot('Year').sum()

In [170]:
counted.show()

+------+----+----+----+----+----+
|    ID|2015|2016|2017|2018|2019|
+------+----+----+----+----+----+
|163768|null|null|null|   7|null|
| 29549|null|null|   2|null|null|
| 92027|null|null|null|   4|null|
| 22254|null|   4|null|null|null|
| 16504|null|null|null|   1|null|
| 93924|null|   1|null|null|null|
| 84188|null|null|   1|null|null|
|  6194|null|   5|null|null|null|
|179486|   1|null|null|null|null|
| 55371|null|null|null|   3|null|
| 23318|   1|null|null|null|null|
| 12529|null|null|null|   1|null|
| 34635|null|null|   1|   1|null|
|100227|null|   1|null|null|null|
| 20158|   1|null|null|null|null|
| 16974|null|null|null|   1|null|
| 46870|   1|   1|null|null|null|
|164905|null|null|   1|null|null|
|  6366|null|null|null|   2|null|
| 65979|  12|null|null|null|null|
+------+----+----+----+----+----+
only showing top 20 rows



In [171]:
counted = counted.na.fill(0)
counted.show()


+------+----+----+----+----+----+
|    ID|2015|2016|2017|2018|2019|
+------+----+----+----+----+----+
|163768|   0|   0|   0|   7|   0|
| 29549|   0|   0|   2|   0|   0|
| 92027|   0|   0|   0|   4|   0|
| 22254|   0|   4|   0|   0|   0|
| 16504|   0|   0|   0|   1|   0|
| 93924|   0|   1|   0|   0|   0|
| 84188|   0|   0|   1|   0|   0|
|  6194|   0|   5|   0|   0|   0|
|179486|   1|   0|   0|   0|   0|
| 55371|   0|   0|   0|   3|   0|
| 23318|   1|   0|   0|   0|   0|
| 12529|   0|   0|   0|   1|   0|
| 34635|   0|   0|   1|   1|   0|
|100227|   0|   1|   0|   0|   0|
| 20158|   1|   0|   0|   0|   0|
| 16974|   0|   0|   0|   1|   0|
| 46870|   1|   1|   0|   0|   0|
|164905|   0|   0|   1|   0|   0|
|  6366|   0|   0|   0|   2|   0|
| 65979|  12|   0|   0|   0|   0|
+------+----+----+----+----+----+
only showing top 20 rows



In [172]:
OLS = counted.groupBy().sum().collect()
OLS

[Row(sum(2015)=4378, sum(2016)=1155, sum(2017)=1324, sum(2018)=6194, sum(2019)=11)]

### Sums per year

In [200]:
years = ['2015','2016','2017','2018','2019']
for i in range(5):
    print(f'Year {years[i]}, sum: {OLS[0][i]}')

Year 2015, sum: 4378
Year 2016, sum: 1155
Year 2017, sum: 1324
Year 2018, sum: 6194
Year 2019, sum: 11


In [182]:
import statsmodels.api as sm

In [183]:
y = [ i for i in OLS[0]]

In [189]:
model = sm.OLS(y, list(range(5)))
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.277
Model:                            OLS   Adj. R-squared (uncentered):              0.096
Method:                 Least Squares   F-statistic:                              1.530
Date:                Wed, 20 May 2020   Prob (F-statistic):                       0.284
Time:                        02:58:41   Log-Likelihood:                         -47.062
No. Observations:                   5   AIC:                                      96.12
Df Residuals:                       4   BIC:                                      95.73
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

/Users/ronaldadomako/anaconda3/lib/python3.7/site-packages/statsmodels/stats/stattools.py:71: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  "samples were given." % int(n), ValueWarning)


In [194]:
results.params[0]

747.6333333333333

# Match percentage/ 100%

In [204]:
total/ 25000

0.52248

In [ ]:
OL